In [1]:
import pandas as pd
import numpy as np

import os
import itertools
from collections import Counter
import re
import string


import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

import spacy

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<img src="images/ner.png">

In [2]:
neg_reviews = pd.read_csv('data/neg_reviews.csv')
neg_reviews.head()


,reviews,tokens,sentiment
0,BA cancelled my flight home to Heathrow on Dec...,"['cancelled', 'home', 'heathrow', 'dec', '19th...",0
1,"BA cancelled my flight home, the last flight o...","['cancelled', 'home', 'last', 'day', 'heathrow...",0
2,"Turned up 3.5 hours in advance, Terminal 5 at ...","['turned', '3.5', 'hours', 'advance', 'termina...",0
3,Boarding – at gate at LGW they called Group 1 ...,"['boarding', '–', 'gate', 'lgw', 'called', 'gr...",0
4,Missing baggage customer service was the worst...,"['missing', 'baggage', 'customer', 'service', ...",0


IOB labeling is a technique used in natural language processing to annotate tokens in text data with their corresponding entity types. IOB stands for "Inside, Outside, Beginning," which refers to the labels given to each token in a sequence.

The IOB scheme uses three types of labels:

    B-xxx (Beginning): This label is used to indicate the first token in an entity.
    I-xxx (Inside): This label is used to indicate any token inside an entity except for the first one.
    O (Outside): This label is used to indicate that a token is not part of any entity.

For example, consider the following sentence: "John Smith works at ABC Corp in New York." Here, the tokens "John" and "Smith" are part of the entity "person," while "ABC" and "New York" are part of the entity "organization" and "location," respectively. The IOB labels for this sentence would be:

    John (B-person)
    Smith (I-person)
    works (O)
    at (O)
    ABC (B-organization)
    Corp (I-organization)
    in (O)
    New (B-location)
    York (I-location)

IOB labeling is often used as a preprocessing step for named entity recognition (NER) tasks, where the goal is to automatically identify and classify named entities in text data.

In [3]:
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(string.punctuation)
stpwrd.extend(['like','have','any','could','sister','would','said','asked','london'])

In [ ]:
def tokenize(txt):
    tokens=[word for word in nltk.word_tokenize(txt) if len(word)>3]
    return tokens

In [7]:
import spacy
from spacy import displacy,tokenizer

# load the pre-trained NER model
nlp = spacy.load("en_core_web_sm")

# sample text to analyze
text = "Apple is looking at buying U.K. startup for $1 billion"



In [8]:
def extract_named_entities(text):
    doc = nlp(text)
    named_entities = []
    for ent in doc.ents:
        named_entities.append(ent.text)
    return named_entities

neg_reviews['named_entities'] = neg_reviews['reviews'].apply(extract_named_entities)
neg_reviews.head()

,reviews,tokens,sentiment,named_entities
0,BA cancelled my flight home to Heathrow on Dec...,"['cancelled', 'home', 'heathrow', 'dec', '19th...",0,"[BA, Heathrow, Dec 19th, Iceland, only 4 hours..."
1,"BA cancelled my flight home, the last flight o...","['cancelled', 'home', 'last', 'day', 'heathrow...",0,"[BA, the day, Heathrow, London City, 3 hours, ..."
2,"Turned up 3.5 hours in advance, Terminal 5 at ...","['turned', '3.5', 'hours', 'advance', 'termina...",0,"[3.5 hours, 5, London Heathrow, 100s, only 5, ..."
3,Boarding – at gate at LGW they called Group 1 ...,"['boarding', '–', 'gate', 'lgw', 'called', 'gr...",0,"[LGW, Group 1, World Traveller Plus & Group, 3..."
4,Missing baggage customer service was the worst...,"['missing', 'baggage', 'customer', 'service', ...",0,"[Phoenix, Sunday, evening, Monday, three more ..."


In [11]:
# Process the first review
doc = nlp(neg_reviews['reviews'][0])
displacy.render(doc, style="ent", jupyter=True)